<a href="https://colab.research.google.com/github/ScorcaF/imitation/blob/master/MBRL_GAILcartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train an Agent using Generative Adversarial Imitation Learning

The idea of generative adversarial imitation learning is to train a discriminator network to distinguish between expert trajectories and learner trajectories.
The learner is trained using a traditional reinforcement learning algorithm such as PPO and is rewarded for trajectories that make the discriminator think that it was an expert trajectory.

In [1]:
%%capture 
%%bash

git clone http://github.com/ScorcaF/imitation
cd imitation && git checkout 0861607f146457e3e086ee91c362c39aeac1d8c4
pip install -e .

pip install mbrl
pip install omegaconf
apt-get install swig
pip install matplotlib==3.1.1
# install required system dependencies
apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

pip3 install box2d-py
pip3 install gym[Box_2D]
pip install stable_baselines3


# git clone https://github.com/ScorcaF/mbrl-lib.git
# pip install -e ".[dev]"
# pip install imitation

In [2]:
import pyvirtualdisplay

_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

_display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
_ = _display.start()

As usual, we first need an expert. 
Note that we now use a variant of the CartPole environment from the seals package, which has fixed episode durations. Read more about why we do this [here](https://imitation.readthedocs.io/en/latest/guide/variable_horizon.html).

In [21]:
%%capture 
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
import gym
import mbrl.env.cartpole_continuous as cartpole_env


env = cartpole_env.CartPoleEnv()

expert = PPO(
    policy=MlpPolicy,
    env=env,
    seed=0)
expert.learn(100_000)  # Note: set to 100000 to train a proficient expert

We generate some expert trajectories, that the discriminator needs to distinguish from the learner's trajectories.

In [22]:
%%capture 
%cd imitation/src
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.vec_env import DummyVecEnv
%cd -

rollouts = rollout.rollout(
    expert,
    DummyVecEnv([lambda: RolloutInfoWrapper(env)] * 5),
    rollout.make_sample_until(min_timesteps=None, min_episodes=60),
)

In [ ]:
%cd imitation/src
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
%cd -

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

import gym



venv = DummyVecEnv([lambda: env] )
learner = PPO(
    env=venv,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0003,
    n_epochs=10,
)
reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=2048,
    n_disc_updates_per_round=4,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
    allow_variable_horizon=True #READ THE DOCS-------------------------------------------
)

learner_rewards_before_training, _ = evaluate_policy(
    learner, venv, 100, return_episode_rewards=True
)
gail_trainer.train(300_000)  # Note: set to 300000 for better results
learner_rewards_after_training, _ = evaluate_policy(
    learner, venv, 100, return_episode_rewards=True
)

/content/imitation/src
/content
Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/guide/variable_horizon.html for more information.


/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,
round:   0%|          | 0/146 [00:00<?, ?it/s]

--------------------------------------
| raw/                        |      |
|    gen/time/fps             | 780  |
|    gen/time/iterations      | 1    |
|    gen/time/time_elapsed    | 2    |
|    gen/time/total_timesteps | 2048 |
--------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.327    |
|    disc/disc_acc_expert             | 0.0156   |
|    disc/disc_acc_gen                | 0.638    |
|    disc/disc_entropy                | 0.69     |
|    disc/disc_loss                   | 0.737    |
|    disc/disc_proportion_expert_pred | 0.189    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
--------------------------------------------------
--------------------------------------------------
| raw/       

round:   1%|          | 1/146 [00:03<09:15,  3.83s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_rew_wrapped_mean | 16.9        |
|    gen/time/fps                    | 785         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 2           |
|    gen/time/total_timesteps        | 4096        |
|    gen/train/approx_kl             | 0.009815651 |
|    gen/train/clip_fraction         | 0.0854      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -1.43       |
|    gen/train/explained_variance    | 0.00182     |
|    gen/train/learning_rate         | 0.0003      |
|    gen/train/loss                  | 2.66        |
|    gen/train/n_updates             | 10          |
|    gen/train/policy_gradient_loss  | -0.0128     |
|    gen/train/std                   | 1.02        |
|    gen/train/value_loss            | 21.7        |
----------------------------------------------

...

round:  99%|█████████▊| 144/146 [09:09<00:07,  3.86s/it]

--------------------------------------------------
| raw/                               |           |
|    gen/rollout/ep_rew_wrapped_mean | 497       |
|    gen/time/fps                    | 781       |
|    gen/time/iterations             | 1         |
|    gen/time/time_elapsed           | 2         |
|    gen/time/total_timesteps        | 296960    |
|    gen/train/approx_kl             | 0.0087937 |
|    gen/train/clip_fraction         | 0.107     |
|    gen/train/clip_range            | 0.2       |
|    gen/train/entropy_loss          | -1.45     |
|    gen/train/explained_variance    | 0.327     |
|    gen/train/learning_rate         | 0.0003    |
|    gen/train/loss                  | 4.35      |
|    gen/train/n_updates             | 1440      |
|    gen/train/policy_gradient_loss  | -0.00535  |
|    gen/train/std                   | 1.03      |
|    gen/train/value_loss            | 21.3      |
--------------------------------------------------
-------------------------------

round:  99%|█████████▉| 145/146 [09:13<00:03,  3.86s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_rew_wrapped_mean | 497         |
|    gen/time/fps                    | 780         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 2           |
|    gen/time/total_timesteps        | 299008      |
|    gen/train/approx_kl             | 0.010147174 |
|    gen/train/clip_fraction         | 0.14        |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -1.44       |
|    gen/train/explained_variance    | 0.477       |
|    gen/train/learning_rate         | 0.0003      |
|    gen/train/loss                  | 1.26        |
|    gen/train/n_updates             | 1450        |
|    gen/train/policy_gradient_loss  | -0.00443    |
|    gen/train/std                   | 1.02        |
|    gen/train/value_loss            | 2.59        |
----------------------------------------------

round: 100%|██████████| 146/146 [09:17<00:00,  3.82s/it]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print(np.mean(learner_rewards_after_training))
print(np.mean(learner_rewards_before_training))

plt.hist(
    [learner_rewards_before_training, learner_rewards_after_training],
    label=["untrained", "trained"],
)
plt.legend()
plt.show()

In [ ]:
%%capture 
from IPython import display
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
import omegaconf
import gym

import mbrl.env.reward_fns as reward_fns
import mbrl.env.termination_fns as termination_fns
import mbrl.models as models
import mbrl.planning as planning
import mbrl.util.common as common_util
import mbrl.util as util




%load_ext autoreload
%autoreload 2

mpl.rcParams.update({"font.size": 16})

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

seed = 0
# env = cartpole_env.CartPoleEnv()
env = gail_trainer.venv_train
env.seed(seed)
rng = np.random.default_rng(seed=0)
generator = torch.Generator(device=device)
generator.manual_seed(seed)
obs_shape = env.observation_space.shape
act_shape = env.action_space.shape

# This functions allows the model to evaluate the true rewards given an observation 
# reward_fn = reward_fns.cartpole
# # This function allows the model to know if an observation should make the episode end
term_fn = termination_fns.cartpole

In [ ]:
%%capture 
trial_length = 200
num_trials = 10
ensemble_size = 1

# Everything with "???" indicates an option with a missing value.
# Our utility functions will fill in these details using the 
# environment information
cfg_dict = {
    # dynamics model configuration
    "dynamics_model": {
        "model": 
        {
            "_target_": "mbrl.models.GaussianMLP",
            "device": device,
            "num_layers": 3,
            "ensemble_size": ensemble_size,
            "hid_size": 200,
            "in_size": "???",
            "out_size": "???",
            "deterministic": False,
            "propagation_method": "fixed_model",
            # can also configure activation function for GaussianMLP
            "activation_fn_cfg": {
                "_target_": "torch.nn.LeakyReLU",
                "negative_slope": 0.01
            }
        }
    },
    # options for training the dynamics model
    "algorithm": {
        "learned_rewards": False,
        "target_is_delta": True,
        "normalize": True,
    },
    # these are experiment specific options
    "overrides": {
        "trial_length": trial_length,
        "num_steps": num_trials * trial_length,
        "model_batch_size": 32,
        "validation_ratio": 0.05
    }
}
cfg = omegaconf.OmegaConf.create(cfg_dict)


# Create a 1-D dynamics model for this environment
dynamics_model = common_util.create_one_dim_tr_model(cfg, obs_shape, act_shape)

# Create a gym-like environment to encapsulate the model
model_env = models.ModelEnv(env, dynamics_model, term_fn,  generator=generator)


replay_buffer = common_util.create_replay_buffer(cfg, obs_shape, act_shape, rng=rng)

In [ ]:
common_util.rollout_agent_trajectories(
    env,
    trial_length, # initial exploration steps
    planning.RandomAgent(env),
    {}, # keyword arguments to pass to agent.act()
    replay_buffer=replay_buffer,
    trial_length=trial_length)

In [ ]:
%%capture 


agent_cfg = omegaconf.OmegaConf.create({
    # this class evaluates many trajectories and picks the best one
    "_target_": "mbrl.planning.TrajectoryOptimizerAgent",
    "planning_horizon": 15,
    "replan_freq": 1,
    "verbose": False,
    "action_lb": "???",
    "action_ub": "???",
    # this is the optimizer to generate and choose a trajectory
    "optimizer_cfg": {
        "_target_": "mbrl.planning.CEMOptimizer",
        "num_iterations": 5,
        "elite_ratio": 0.1,
        "population_size": 500,
        "alpha": 0.1,
        "device": device,
        "lower_bound": "???",
        "upper_bound": "???",
        "return_mean_elites": True,
    }
})

discriminator = gail_trainer._reward_net

# Need to insert D-net in reward_fn or model env 

# reward_fn = reward_fns.discriminator
model_env = models.ModelEnv(discriminator, env, dynamics_model, term_fn, generator=generator)

agent = planning.create_trajectory_optim_agent_for_model(
    model_env,
    agent_cfg,
    num_particles=20
)

In [ ]:
model_env.reset(np.random.random((4,4)))["obs"]

In [ ]:
train_losses = []
val_scores = []

def train_callback(_model, _total_calls, _epoch, tr_loss, val_score, _best_val):
    train_losses.append(tr_loss)
    val_scores.append(val_score.mean().item())   # this returns val score per ensemble model

def update_axes(_axs, _frame, _text, _trial, _steps_trial, _all_rewards, force_update=False):
    if not force_update and (_steps_trial % 10 != 0):
        return
    _axs[0].imshow(_frame)
    _axs[0].set_xticks([])
    _axs[0].set_yticks([])
    _axs[1].clear()
    _axs[1].set_xlim([0, num_trials + .1])
    _axs[1].set_ylim([0, 200])
    _axs[1].set_xlabel("Trial")
    _axs[1].set_ylabel("Trial reward")
    _axs[1].plot(_all_rewards, 'bs-')
    _text.set_text(f"Trial {_trial + 1}: {_steps_trial} steps")
    display.display(plt.gcf())  
    display.clear_output(wait=True)
    plt.savefig("mygraph.png")

#  Create a trainer for the model
model_trainer = models.ModelTrainer(dynamics_model, optim_lr=1e-3, weight_decay=5e-5)
num_trials = 10
# Create visualization objects
fig, axs = plt.subplots(1, 2, figsize=(14, 3.75), gridspec_kw={"width_ratios": [1, 1]})
ax_text = axs[0].text(300, 50, "")
    
# Main PETS loop
all_rewards = [0]
for trial in range(num_trials):
    obs = env.reset()    
    agent.reset()
    
    done = False
    total_reward = 0.0
    steps_trial = 0
    update_axes(axs, env.render(mode="rgb_array"), ax_text, trial, steps_trial, all_rewards)
    while not done:
        # --------------- Model Training -----------------
        if steps_trial == 0:
            dynamics_model.update_normalizer(replay_buffer.get_all())  # update normalizer stats --> all the input arrays must have same number of dimensions, but the array at index 0 has 4 dimension(s) and the array at index 1 has 2 dimension(s)
            
            dataset_train, dataset_val = common_util.get_basic_buffer_iterators(
                replay_buffer,
                batch_size=cfg.overrides.model_batch_size,
                val_ratio=cfg.overrides.validation_ratio,
                ensemble_size=ensemble_size,
                shuffle_each_epoch=True,
                bootstrap_permutes=False,  # build bootstrap dataset using sampling with replacement
            )
                
            model_trainer.train(
                dataset_train, 
                dataset_val=dataset_val, 
                num_epochs=50, 
                patience=10, 
                callback=train_callback,
            )

        # --- Doing env step using the agent and adding to model dataset ---
        next_obs, reward, done, _ = common_util.step_env_and_add_to_buffer(
            env, obs, agent, {}, replay_buffer)
            
        update_axes(axs, env.render(mode="rgb_array"), ax_text, trial, steps_trial, all_rewards)
        
        obs = next_obs
        total_reward += reward
        steps_trial += 1
        
        if steps_trial == trial_length:
            break
    
    all_rewards.append(total_reward)

update_axes(axs, env.render(mode="rgb_array"), ax_text, trial, steps_trial, all_rewards, force_update=True)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(2, 1, figsize=(12, 10))
ax[0].plot(train_losses)
ax[0].set_xlabel("Total training epochs")
ax[0].set_ylabel("Training loss (avg. NLL)")
ax[1].plot(val_scores)
ax[1].set_xlabel("Total training epochs")
ax[1].set_ylabel("Validation score (avg. MSE)")
plt.show()

#Code for printing sequences

In [18]:
obs_seq, rew_seq, act_seq = common_util.rollout_model_env(
    model_env=model_env,
    initial_obs=env.reset().squeeze(),
    agent=agent,
    num_samples=1,
)

#Code for tests with MBRL inserted into GAIL

In [ ]:
%%capture 
%cd imitation/src
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
%cd -

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv


#Need to manage vec envs
venv = DummyVecEnv([lambda: env] * 1)

reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)

model_trainer = models.ModelTrainer(dynamics_model, optim_lr=1e-3, weight_decay=5e-5)

# RuntimeError: BufferingWrapper reset() before samples were accessed, n_stored = 200*times_execution
# common_util.rollout_agent_trajectories(
#     env,
#     trial_length, # initial exploration steps
#     planning.RandomAgent(env),
#     {}, # keyword arguments to pass to agent.act()
#     replay_buffer=replay_buffer,
#     trial_length=trial_length)


gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=2048,
    n_disc_updates_per_round=4,
    venv=venv,
    gen_algo=agent,
    reward_net=reward_net,
    cfg=cfg,
    model_trainer=model_trainer,
    dynamics_model=dynamics_model,
    replay_buffer=replay_buffer,
    gen_train_timesteps = 2000,
    allow_variable_horizon=True #https://imitation.readthedocs.io/en/latest/guide/variable_horizon.html
)




In [ ]:
gail_trainer.train(20000)  # Note: set to 300000 for better results

# ValueError: Wrong data shape for acts ------- acts array monodimensional, other bidemensional
# Need to reshape actions for single-dim act space: added "action = action.reshape(-1,1)"

round:   0%|          | 0/10 [02:38<?, ?it/s]

gen_trajs [TrajectoryWithRew(obs=array([[ 3.82382199e-02, -2.71885116e-02,  8.95104650e-03,
        -1.65020972e-02],
       [ 3.76944467e-02,  1.19631752e-01,  8.62100441e-03,
        -2.34092101e-01],
       [ 4.00870815e-02, -4.49558208e-03,  3.93916247e-03,
        -4.53734733e-02],
       [ 3.99971716e-02,  8.10967535e-02,  3.03169270e-03,
        -1.72602862e-01],
       [ 4.16191071e-02, -2.32035737e-03, -4.20364464e-04,
        -4.65864614e-02],
       [ 4.15727012e-02, -1.52969480e-01, -1.35209365e-03,
         1.79263622e-01],
       [ 3.85133103e-02, -2.38789730e-02,  2.23317859e-03,
        -1.47694843e-02],
       [ 3.80357318e-02,  2.75642928e-02,  1.93778903e-03,
        -9.12776366e-02],
       [ 3.85870151e-02, -1.32560745e-01,  1.12236303e-04,
         1.49479181e-01],
       [ 3.59358005e-02, -2.45503664e-01,  3.10181989e-03,
         3.18926543e-01],
       [ 3.10257282e-02, -2.01829851e-01,  9.48035065e-03,
         2.54328072e-01],
       [ 2.69891303e-02, -3.6757

AttributeError: ignored

In [ ]:
#changed "action.reshape(-1,1)" in trajectory_opt.py
agent.act(gail_trainer.venv_train.reset())

plan [[ 0.2782188 ]
 [ 0.02829593]
 [ 0.05018038]
 [-0.14561155]
 [-0.1997795 ]
 [-0.1083254 ]
 [-0.07284492]
 [-0.31208798]
 [ 0.16547456]
 [-0.12820692]
 [-0.12892792]
 [ 0.03909729]
 [-0.06817953]
 [ 0.0910966 ]
 [-0.04198803]]
self.actions_to_use  [array([0.2782188], dtype=float32)]


array([[0.2782188]], dtype=float32)

When we look at the histograms of rewards before and after learning, we can see that the learner is not perfect yet, but it made some progress at least.
If not, just re-run the above cell.